# Versão 4 dos testes realizados para o modelo


In [31]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import re

In [32]:
# Lê as propostas com o Pandas
propostas = pd.read_csv("/home/leandro/Documentos/07_ProcessamentoLinguagemNatural/production/data_extraction/propostas.csv")

propostas  # Impressão do DataFrame de propostas

,Categoria,Texto
0,Turismo,Turismo: esse é o Destino. <p><strong>Objetivo...
1,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia. <p><stron...
2,Agricultura e Pecuária,Agropecuária Sustentável. <p>Objetivo: Contrib...
3,Saúde,Atenção Primária à Saúde. <p>Fortalecer a Aten...
4,Saúde,Atenção Especializada à Saúde. <p>Ampliar o ac...
...,...,...
8431,Saúde,Microchipagem de animais de companhia. Microch...
8432,Secretaria Geral da Presidência da República,"Direito a atendimento eficiente, é um direito..."
8433,Previdência Social,Rever Reforma da Previdência. Rever a reforma ...
8434,Meio Ambiente e Mudança do Clima,Preservação da Serra do Espinhaço. A Serra do ...


In [33]:
# Expressões regulares para remoção de poluição de dados nos textos 
# REGX_URL = r"https?://[A-Za-z0-9./\-]+" # Regex for URLs
REGX_HTML = r"<[^<]+?>" # Regex for HTML tags
REGX_ENDING = r'Órgão Responsável:.+' # Regex for the part of the text

In [34]:
# Função para pré-processamento
def preprocessing(text):
  # text = text.lower()

  text = re.sub(REGX_HTML, '', text)  # Removendo tags HTML
  # text = re.sub(REGX_URL, '', text) # Revomendo URLs
  text = re.sub(REGX_ENDING, '', text)

  # tokens = [t.lemma_ for t in nlp(text) if t not in STOP_WORDS and not t.is_punct]

  return text

#  Ajustando textos da coluna Corpo
propostas['Texto'] = propostas['Texto'].apply(preprocessing)
propostas.head()

,Categoria,Texto
0,Turismo,Turismo: esse é o Destino. Objetivo: Posiciona...
1,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia. Objetivo:...
2,Agricultura e Pecuária,Agropecuária Sustentável. Objetivo: Contribuir...
3,Saúde,Atenção Primária à Saúde. Fortalecer a Atenção...
4,Saúde,Atenção Especializada à Saúde. Ampliar o acess...


In [35]:
print(propostas.Categoria.value_counts())

Categoria
Saúde                                                             1368
Educação                                                          1261
Direitos Humanos e Cidadania                                       528
Meio Ambiente e Mudança do Clima                                   497
Transportes                                                        473
Desenvolvimento e Assistência Social, Família e Combate à Fome     418
Trabalho e Emprego                                                 377
Justiça e Segurança Pública                                        342
Cidades                                                            294
Gestão e Inovação em Serviços Públicos                             284
Desenvolvimento Agrário e Agricultura Familiar                     255
Previdência Social                                                 249
Cultura                                                            224
Mulheres                                                           

In [36]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from sklearn.model_selection import train_test_split

# Dividir dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(propostas['Texto'],propostas["Categoria"],stratify=propostas.Categoria, test_size=0.2, random_state=42)

# Criar vetorizador TF-IDF com n-grams
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Aqui usamos bigrams
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Treinar SVM
svm_model = SVC(kernel='linear')  # Pode usar outros kernels também
svm_model.fit(X_train_tfidf, y_train)

# Prever e avaliar
y_pred = svm_model.predict(X_test_tfidf)
report = classification_report(y_test, y_pred, zero_division=0)


print(report)


                                                                precision    recall  f1-score   support

                                      Advocacia-Geral da União       0.00      0.00      0.00         3
                                        Agricultura e Pecuária       0.67      0.10      0.17        21
                                                 Banco Central       1.00      0.25      0.40         4
                                                    Casa Civil       0.00      0.00      0.00         7
                                                       Cidades       0.44      0.37      0.40        59
                                Ciência, Tecnologia e Inovação       0.55      0.19      0.28        32
                                            Comunicação Social       0.00      0.00      0.00         9
                                                  Comunicações       0.67      0.22      0.33         9
                                  Controladoria-Geral da União 

In [37]:
y_train.value_counts()


Categoria
Saúde                                                             1094
Educação                                                          1009
Direitos Humanos e Cidadania                                       422
Meio Ambiente e Mudança do Clima                                   397
Transportes                                                        378
Desenvolvimento e Assistência Social, Família e Combate à Fome     334
Trabalho e Emprego                                                 301
Justiça e Segurança Pública                                        273
Cidades                                                            235
Gestão e Inovação em Serviços Públicos                             227
Desenvolvimento Agrário e Agricultura Familiar                     204
Previdência Social                                                 199
Cultura                                                            179
Mulheres                                                           

In [38]:
y_test.value_counts()

Categoria
Saúde                                                             274
Educação                                                          252
Direitos Humanos e Cidadania                                      106
Meio Ambiente e Mudança do Clima                                  100
Transportes                                                        95
Desenvolvimento e Assistência Social, Família e Combate à Fome     84
Trabalho e Emprego                                                 76
Justiça e Segurança Pública                                        69
Cidades                                                            59
Gestão e Inovação em Serviços Públicos                             57
Desenvolvimento Agrário e Agricultura Familiar                     51
Previdência Social                                                 50
Cultura                                                            45
Mulheres                                                           37
Ciência, T

In [39]:
relacoes_ext = propostas.loc[propostas['Categoria'].isin(['Relações Exteriores'])]
relacoes_ext

,Categoria,Texto
173,Relações Exteriores,Passaporte mais rapido e menos burocratico . S...
1158,Relações Exteriores,Aprimoramento da plataforma Concórdia para fin...
2947,Relações Exteriores,Fortalecimento do Mercosul e do BRICS através...
2992,Relações Exteriores,"Entretenimento . Promover curtas, longas e sér..."
3709,Relações Exteriores,Criação de bolsas de aperfeiçoamento profissio...
4447,Relações Exteriores,Cooperação internacional para combater o tráfi...
6156,Relações Exteriores,Taxação para produção e exportação de alimento...
6536,Relações Exteriores,Criação do Conselho Nacional de Política Exter...
7035,Relações Exteriores,"Estruturação dos escritórios, diretorias e coo..."
7326,Relações Exteriores,Implementação de uma política externa feminist...


In [40]:
relacoes_ext['Texto'][3709]

'Criação de bolsas de aperfeiçoamento profissional - não acadêmicas - para cidadãos de países em desenvolvimento em Universidades e Institutos Federais. A criação de bolsas não acadêmicas de aperfeiçoamento profissional para cidadãos de países em desenvolvimento poderá fortalecer os vínculos sul-sul e contribuir para o desenvolvimento global.'

# Realizando a criacao de novas propostas e verificando a performance no modelo



In [41]:
# Lê as propostas com o Pandas

simuladas = pd.read_csv("propostas_simuladas.csv")
simuladas  # Impressão do DataFrame de propostas

,Categoria,Proposta
0,Casa Civil,É ótimo ver a conscientização sobre igualdade ...
1,Casa Civil,Tornar a Casa Civil mais ecológica e sustentáv...
2,Casa Civil,Criar um programa de parcerias com startups na...
3,Casa Civil,Vamos estabelecer metas ambiciosas para reduzi...
4,Casa Civil,Vamos criar um comitê de ética interno na Casa...
...,...,...
366,Comunicações,O serviço postal desempenha um papel essencial...
367,Comunicações,A imprensa local desempenha um papel vital na ...
368,Comunicações,Fusões e aquisições no setor de mídia podem te...
369,Comunicações,"Para capacitar os cidadãos na era digital, é c..."


In [42]:
propostas = pd.DataFrame(propostas)
propostas_simuladas = pd.DataFrame(simuladas)
novas_propostas = pd.concat([propostas,propostas_simuladas],ignore_index=True)

In [43]:
print(novas_propostas.Categoria.value_counts())

Categoria
Saúde                                                             1368
Educação                                                          1261
Direitos Humanos e Cidadania                                       528
Meio Ambiente e Mudança do Clima                                   497
Transportes                                                        473
Desenvolvimento e Assistência Social, Família e Combate à Fome     418
Trabalho e Emprego                                                 377
Justiça e Segurança Pública                                        342
Cidades                                                            294
Gestão e Inovação em Serviços Públicos                             284
Desenvolvimento Agrário e Agricultura Familiar                     255
Previdência Social                                                 249
Cultura                                                            224
Mulheres                                                           

In [44]:
import json

# Dividir dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(novas_propostas['Texto'].values.astype('U'),novas_propostas["Categoria"].values.astype('U'),stratify=novas_propostas.Categoria, test_size=0.2, random_state=42)

# Criar vetorizador TF-IDF com n-grams
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Aqui usamos bigrams
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Treinar SVM
svm_model = SVC(kernel='linear')  # Pode usar outros kernels também
svm_model.fit(X_train_tfidf, y_train)

# Prever e avaliar
y_pred = svm_model.predict(X_test_tfidf)
report = classification_report(y_test, y_pred, zero_division=0)

# Salvando relatório de treinamento em um arquivo JSON
json_obj = json.dumps(classification_report(X_test, y_pred, output_dict=True), indent=4)
save_file = open('../model_v4/training_results.json', 'w').write(json_obj)



/home/leandro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/leandro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/leandro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/leandro/.local/lib/python3.10/site-p

In [45]:
print(report)

                                                                precision    recall  f1-score   support

                                      Advocacia-Geral da União       0.00      0.00      0.00        11
                                        Agricultura e Pecuária       0.60      0.14      0.23        21
                                                 Banco Central       1.00      0.09      0.17        11
                                                    Casa Civil       0.10      0.60      0.18        15
                                                       Cidades       0.47      0.37      0.42        59
                                Ciência, Tecnologia e Inovação       0.50      0.25      0.33        32
                                            Comunicação Social       0.00      0.00      0.00        12
                                                  Comunicações       0.00      0.00      0.00        13
                                  Controladoria-Geral da União 

# Corrigindo categorias sem performance


In [46]:
import nltk
from nltk import tokenize
from nltk.corpus import stopwords



# realiza a tokenização do texto
token_espaco = tokenize.WhitespaceTokenizer()


In [47]:
# Tratamento 1

palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")

frase_processada = list()
for opiniao in novas_propostas.Texto:
    nova_frase = list()
    palavras_texto = token_espaco.tokenize(str(opiniao))
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantes:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))

novas_propostas["tratamento_1"] = frase_processada


In [48]:
# Tratamento 2

from string import punctuation

token_pontuacao = tokenize.WordPunctTokenizer()

pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)

pontuacao_stopwords = pontuacao + palavras_irrelevantes

frase_processada = list()
for opiniao in novas_propostas["tratamento_1"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))

novas_propostas["tratamento_2"] = frase_processada

In [49]:
# tratamento 3

import unidecode 

sem_acentos = [unidecode.unidecode(texto) for texto in novas_propostas["tratamento_2"]]

novas_propostas["tratamento_3"] = sem_acentos

frase_processada = list()
for opiniao in novas_propostas["tratamento_3"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))

novas_propostas["tratamento_3"] = frase_processada



In [50]:
# tratamento 4

stopwords_sem_acento = [unidecode.unidecode(texto) for texto in pontuacao_stopwords]

#deixa todas as letras minusculas
frase_processada = list()
for opiniao in novas_propostas["tratamento_3"]:
    nova_frase = list()
    opiniao = opiniao.lower()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in stopwords_sem_acento:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))

novas_propostas["tratamento_4"] = frase_processada


In [51]:
novas_propostas

,Categoria,Texto,Proposta,tratamento_1,tratamento_2,tratamento_3,tratamento_4
0,Turismo,Turismo: esse é o Destino. Objetivo: Posiciona...,NaN,Turismo: Destino. Objetivo: Posicionar turismo...,Turismo Destino Objetivo Posicionar turismo ve...,Turismo Destino Objetivo Posicionar turismo ve...,turismo destino objetivo posicionar turismo ve...
1,Desenvolvimento Agrário e Agricultura Familiar,Agricultura Familiar e Agroecologia. Objetivo:...,NaN,Agricultura Familiar Agroecologia. Objetivo: F...,Agricultura Familiar Agroecologia Objetivo For...,Agricultura Familiar Agroecologia Objetivo For...,agricultura familiar agroecologia objetivo for...
2,Agricultura e Pecuária,Agropecuária Sustentável. Objetivo: Contribuir...,NaN,Agropecuária Sustentável. Objetivo: Contribuir...,Agropecuária Sustentável Objetivo Contribuir d...,Agropecuaria Sustentavel Objetivo Contribuir d...,agropecuaria sustentavel objetivo contribuir d...
3,Saúde,Atenção Primária à Saúde. Fortalecer a Atenção...,NaN,Atenção Primária Saúde. Fortalecer Atenção Pri...,Atenção Primária Saúde Fortalecer Atenção Prim...,Atencao Primaria Saude Fortalecer Atencao Prim...,atencao primaria saude fortalecer atencao prim...
4,Saúde,Atenção Especializada à Saúde. Ampliar o acess...,NaN,Atenção Especializada Saúde. Ampliar acesso aç...,Atenção Especializada Saúde Ampliar acesso açõ...,Atencao Especializada Saude Ampliar acesso aco...,atencao especializada saude ampliar acesso aco...
...,...,...,...,...,...,...,...
8802,Comunicações,NaN,O serviço postal desempenha um papel essencial...,nan,nan,nan,nan
8803,Comunicações,NaN,A imprensa local desempenha um papel vital na ...,nan,nan,nan,nan
8804,Comunicações,NaN,Fusões e aquisições no setor de mídia podem te...,nan,nan,nan,nan
8805,Comunicações,NaN,"Para capacitar os cidadãos na era digital, é c...",nan,nan,nan,nan


In [52]:
# Tratamento 5 Lematizacao com spacy 

import spacy
from spacy.lang.pt.stop_words import STOP_WORDS

nlp = spacy.load("pt_core_news_sm") 

frase_processada = list()
for opiniao in novas_propostas["tratamento_4"]:
    nova_frase = list()
    doc = nlp(opiniao)
    for token in doc:
        nova_frase.append(token.lemma_)
    frase_processada.append(' '.join(nova_frase))

novas_propostas["tratamento_5"] = frase_processada


In [67]:
import json

# Dividir dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(novas_propostas['Texto'].values.astype('U'),novas_propostas["Categoria"].values.astype('U'),stratify=novas_propostas.Categoria, test_size=0.2, random_state=42)

# Criar vetorizador TF-IDF com n-grams
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Aqui usamos bigrams
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Treinar SVM
svm_model = SVC(kernel='linear')  # Pode usar outros kernels também
svm_model.fit(X_train_tfidf, y_train)

# Prever e avaliar
y_pred = svm_model.predict(X_test_tfidf)
report = classification_report(y_test, y_pred, zero_division=0)



In [54]:
print(report)

                                                                precision    recall  f1-score   support

                                      Advocacia-Geral da União       0.00      0.00      0.00        11
                                        Agricultura e Pecuária       0.60      0.14      0.23        21
                                                 Banco Central       1.00      0.09      0.17        11
                                                    Casa Civil       0.10      0.60      0.18        15
                                                       Cidades       0.47      0.37      0.42        59
                                Ciência, Tecnologia e Inovação       0.50      0.25      0.33        32
                                            Comunicação Social       0.00      0.00      0.00        12
                                                  Comunicações       0.00      0.00      0.00        13
                                  Controladoria-Geral da União 

In [71]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resempled = smote.fit_resample(X_train_tfidf, y_train)

svm_model = SVC(kernel='linear')
svm_model.fit(X_train_resampled, y_train_resempled)

y_pred = svm_model.predict(X_test_tfidf)
report = classification_report(y_test, y_pred, zero_division=0)

In [73]:
print(report)

                                                                precision    recall  f1-score   support

                                      Advocacia-Geral da União       0.10      0.82      0.18        11
                                        Agricultura e Pecuária       0.38      0.14      0.21        21
                                                 Banco Central       0.50      0.09      0.15        11
                                                    Casa Civil       0.00      0.00      0.00        15
                                                       Cidades       0.41      0.37      0.39        59
                                Ciência, Tecnologia e Inovação       0.38      0.25      0.30        32
                                            Comunicação Social       1.00      0.08      0.15        12
                                                  Comunicações       0.00      0.00      0.00        13
                                  Controladoria-Geral da União 

In [59]:
import numpy as np
from imblearn.under_sampling import RandomUnderSampler, NearMiss, OneSidedSelection

# garantindo que X_train possui 2Dimensão e y_train possui 1Dimensão
X_train = X_train.reshape(-1, 1) if len(X_train.shape) == 1 else X_train
y_train = y_train.flatten()

# Random Undersampler
desired_samples = {
    'Saúde': 100,  # Desired sample count for category 'Saúde'
    'Educação': 100,  # Desired sample count for category 'Educação'
}

rus = RandomUnderSampler(sampling_strategy=desired_samples, random_state=42)


X_rus_res, y_rus_res = rus.fit_resample(X_train, y_train)




In [60]:
print(pd.Series(y_rus_res).value_counts())

Direitos Humanos e Cidadania                                      422
Meio Ambiente e Mudança do Clima                                  397
Transportes                                                       378
Desenvolvimento e Assistência Social, Família e Combate à Fome    334
Trabalho e Emprego                                                302
Justiça e Segurança Pública                                       274
Cidades                                                           235
Gestão e Inovação em Serviços Públicos                            227
Desenvolvimento Agrário e Agricultura Familiar                    204
Previdência Social                                                199
Cultura                                                           179
Mulheres                                                          150
Ciência, Tecnologia e Inovação                                    126
Fazenda                                                           106
Planejamento e Orçam

In [61]:
print(type(X_test))

<class 'numpy.ndarray'>


In [62]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
tfidf_vectorizer = TfidfVectorizer()

X_rus_res_numeric = tfidf_vectorizer.fit_transform(X_rus_res.ravel())
X_test_numeric = tfidf_vectorizer.transform(X_test)  # Use transform instead of fit_transform
y_rus_res_numeric = label_encoder.fit_transform(y_rus_res)
y_test_numeric = label_encoder.transform(y_test)

svc_model = SVC(kernel='linear', random_state=42)

svc_model.fit(X_rus_res_numeric, y_rus_res_numeric)

predictions = svc_model.predict(X_test_numeric)

report = classification_report(y_test_numeric, predictions)
print(report)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.75      0.14      0.24        21
           2       1.00      0.09      0.17        11
           3       0.10      0.60      0.18        15
           4       0.42      0.42      0.42        59
           5       0.39      0.38      0.38        32
           6       1.00      0.08      0.15        12
           7       0.00      0.00      0.00        13
           8       0.00      0.00      0.00        11
           9       0.70      0.73      0.72        45
          10       0.33      0.08      0.13        12
          11       0.55      0.63      0.59        51
          12       0.30      0.42      0.35        84
          13       0.50      0.09      0.15        23
          14       0.17      0.54      0.26       106
          15       0.88      0.50      0.63       252
          16       0.82      0.50      0.62        18
          17       0.57    

/home/leandro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/leandro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/leandro/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
